# Creating command line tools CLI

## The `argparse` module

For System Administrators, command line tools are their daily bread. This section helps to create beautiful command line interfaces.
To make it possible to develop standalone python scripts without dependencies to 3rd party libraries, we will focus on [argparse](https://docs.python.org/3/library/argparse.html), which is available in the python standard library.

<div class="alert alert-block alert-success">
    The 3rd party <a href="https://click.palletsprojects.com/en/7.x/">click module</a> is the most popular among the many that exist, and has some <a href="https://click.palletsprojects.com/en/7.x/why/#why-not-argparse">advantages over argparse</a>, still in many cases, <a href="https://docs.python.org/3/library/argparse.html">argparse</a> will do the job.
</div>

The `argparse` module not only makes it relatively easy to create nice and **extendible command line interfaces**, but also implements many best practices. For example, it automagically creates a **help page** for the command line interface and each of its sub-commands, handles flags in a consistent and predictable way and generates human-friendly errors.

But before we start, we need to understand how command line utilities usually work. They consists of three parts:

```sh
cp -r src/ dest/
```

```
 |  |  └────┴── Arguments
 |  └────────── Option (flag)
 └───────────── Command
```

```sh
git diff -p dev
```

```
 |    |    |  └── Argument
 |    |    └───── Option (flag)
 |    └────────── Sub-command
 └─────────────── Command
```

The *command* is the **verb**, it explains what we would like to do, e.g. `import-datasets` or `copy-logfiles`. You can offer **sub-commands** if you need to. You can completely omit sub-commands, if your utility is doing only one thing.

The *Options* (one, many or none) define more precisely how the utility or the chosen command should behave. Options start with either:
* one dash followed by a letter, e.g. `-a`, `-d`
* two dashes followed by a word,  e.g. `--add`, `--delete`
* options containing two words: `--show-keys`
* Typically most options have a short and a long version
* ... so users can either write `-a` or `--add`.
* Options can be followed by an argument or can act as a switch, like the `--verbose` switch

The *argument*(s) is/are the **subject**, often a filename, but it can be a simple string too. The *argument* follows after each *option*, but can also follow after a *command*.

### Example: Swen's iam tool
✨ ETH gitlab: [gitlab.ethz.ch/sis/tools/ethz-iam-webservice](https://gitlab.ethz.ch/sis/tools/ethz-iam-webservice).

```
Usage: iam [OPTIONS] COMMAND [ARGS]...

  ETHZ IAM command-line tool.

Options:
  -u, --username TEXT  username of ETHZ IAM admin account or IAM_USERNAME env
  --password TEXT      password of ETHZ IAM admin account or IAM_PASSWORD env
  --version            Show the version and exit.
  --help               Show this message and exit.

Commands:
  group    manage security groups
  groups   show all kinds of groups (in LDAP only)
  guest    manage guests
  person   manage persons
  persons  search for persons
  user     manage users and their services
```

## Your first command-line tool

Your first module does a lot of useful things:

* it prompts for your name
* it greets you the many times you want
* it offers a polite version

Look at the code below.

In [ ]:
%%writefile greeter.py
#!/bin/env python3

import argparse

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--count", "-c", default=1, type=int, help="Number of greetings.")
    parser.add_argument("--polite", "-p", action="store_true")
    parser.add_argument("--name", "-n", required=True, help="The person to greet.")
    args = parser.parse_args()
    hello(args.count, args.polite, args.name)


def hello(count, polite, name):
    if polite:
        greeting = "Your Serene Highness"
    else:
        greeting = "Hello"

    for _ in range(count):
        print(f"{greeting} {name}!")


if __name__ == "__main__":
    main()

In [ ]:
!chmod u+x greeter.py

In [ ]:
%%script bash --no-raise-error

./greeter.py

In [ ]:
!./greeter.py --help

In [ ]:
%%script bash --no-raise-error

./greeter.py -n "Chuck Norris"

In [ ]:
%%script bash --no-raise-error

./greeter.py -p --count 2 -n "Chuck Norris"

In [ ]:
%%script bash --no-raise-error

./greeter.py -p --count one # not convertable to an int

### The entry point of your script: `__name__ == '__main__'`

At the end of our small script, you see one of the more confusing things in Python is this pattern:

```python
if __name__ == "__main__":
    main()
```

It's called the **entry point** of a script. It tells the interpreter to call the `hello()` function.

### Exercise 1

* Try to remove the entry point from your [greeter.py](greeter.py) script and then run it with the Python interpreter
```sh
python greeter.py
```
* Observe: what happens?
* try to introduce an obvious error (e.g. `result = 100/0`, either inside or outside the `hello()` function and run it again.
* Can you explain the behaviour?

**Explanation**

The Python interpreter first performs a syntax check on your script. After the syntax check it imports `argparse` and defines the `hello()` function, making the function `hello()` available to the program. Finally it reaches the end of the script and exits.

If any error occurs in the **main section** of the program, the Python interpreter halts and prints out a stacktrace. Non syntax-errors *inside* a function or method are *not* detected, they only occur when a function or method is *executed*.

After the compilation step, we have to tell the Python interpreter what to do next, e.g. execute our function. For this we could also simply add:

```python
hello()
```

However, that `hello()` function call then would **always be executed**, even when we only want to import the script from another script. So we need a way to decide, in which **context** our script was called. That's what the

```python
if __name__ == "__main__":
    hello()
```

statement is for. The special variable `__name__` contains the value `__main__` if the script was the first thing that got started. On the other hand, if our script got `import`ed by another script or module, the `__name__` variable contains the name of the importer. In this case, we probably don't want to run anything automatically.

### Boolean values (flags)

Often we want to adjust the behaviour of our command line tool by providing boolean flags.
An example of a flag is `-p` / `--polite` in the above example. We use `action="store_true"` to define a flag.

Also other so called actions are available in `argparse`: https://docs.python.org/3/library/argparse.html#action

## Arguments

Work almost like options with the difference that they are *positional*. By adding the parameter `nargs="+"`, you can pass one or more values for the argument.

<div class="alert alert-block alert-warning">
⚠️ If the parser expects more than one option, only one of the options can have <code>nargs="+"</code> or <code>nargs="*"</code>, as it eats up all remaining arguments passed to the script.
</div>

| |                   |                   |                     |                    |
|-|-------------------|-------------------|---------------------|--------------------|
| |       ✅          |         ✅        |       ✅           |       ❌          |
| CLI command | `ln -s src dest`  |  `cp src* dest/`  | `chown user files*` |  `cp src* dest*`   |
| | `       ^    ^`   |  `    ^     ^  `  | `        ^    ^`    |  `    ^     ^  `   |
|Arguments | `"src", nargs=1 `|  ` "src", nargs="+" `|`"src", nargs=1`|  ` "src", nargs="+" `|
| | `"dest", nargs=1`|  `"dest", nargs=1 `|`"dest", nargs=+`|  `"dest", nargs="+"`|


### Example

In [ ]:
%%writefile move.py
#!/bin/env python3

import argparse

parser = argparse.ArgumentParser(description="Move file SRC to DST.")
parser.add_argument("src", nargs="+", help="Source file(s)")
parser.add_argument("dest", help="Destination file")

args = parser.parse_args()

for src in args.src:
    print(f"{src} -> {args.dest}")

In [ ]:
!chmod u+x move.py

In [ ]:
%%script bash --no-raise-error

./move.py src-1.txt src-2.txt dest_folder/

### Exercise 2

modify [greeter.py](greeter.py)

- [ ] add a help text for the `--polite` option and test it
- [ ] If the name option is not passed, prompt the userfor a name (remove `required=True`)
- [ ] add an *optional* file argument which raises an error if the path does not exist.
  Instructions:
  * Write a function `def existing_file(path: str) -> pathlib.Path` which takes a path as a `str` as argument,
    creates a `pathlib.Path` object and calls its `exists()` method. If the path exists, return the `pathlib.Path` object, otherwise it raises a `ValueError`.
  * Use `type=existing_file` in `parser.add_argument` (the `type` parameter can be any `Callable` which takes a `str` as an input and canverts it into the desired target type).
- [ ] After parsing the command line options using `args = parser.parse_args()`, if no error happened, open the content of the file and print it after the greeting.

In [ ]:
#!/bin/env python3

import argparse
import pathlib
from typing import Optional

def main(*options: str) -> None:
    parser = argparse.ArgumentParser()
    parser.add_argument("--count", "-c", default=1, type=int, help="Number of greetings.")
    parser.add_argument("--polite", "-p", action="store_true")
    parser.add_argument("--name", "-n", help="The person to greet.")
    parser.add_argument("--file", type=existing_file, help="An optional path to a file which will be printed on greeting.")
    args = parser.parse_args(options)
    if args.name is None:
        name = input("Please enter a name: ")
    else:
        name = args.name
    hello(args.count, args.polite, name, args.file)

def existing_file(path: str) -> pathlib.Path:
    path_obj = pathlib.Path(path)
    if path_obj.exists():
        return path_obj
    raise ValueError(f"Path `{path}` does not exist")


def hello(count: int, polite: bool, name: str, path: Optional[str]) -> None:
    if polite:
        greeting = "Your Serene Highness"
    else:
        greeting = "Hello"

    for _ in range(count):
        print(f"{greeting} {name}!")
    
    if path is not None:
        with open(path) as f:
            print(f.read())

if __name__ == "__main__":
    import sys
    main(*sys.argv[1:])

In [ ]:
main()

In [ ]:
with open("msg.txt", "w") as f:
    f.write("🗯️")
main("--name", "Chuck Norris", "--file", "msg.txt")
main("--name", "Chuck Norris") # file not passed
main("--name", "Chuck Norris", "--file", "not_existing.txt")

## Sub-commands

Sub-commands are a way to conveniently group options for a given task. For example, the [ethz-iam-webservice](https://gitlab.ethz.ch/vermeul/ethz-iam-webservice) CLI offers the `person`, `user` and the `group` *sub-commands*. Below you see some examples how the utility takes a **sub-command** followed by an **argument** and some **options**:

```sh
iam person swen@ethz.ch
iam user vermeul
iam user vermeul --grant-service LDAPS
iam group my_group
iam group my_group -a new_user1 -a new_user2 -r old_user3
iam group my_group --delete
```

### How to implement a sub-command

1. Use the `add_subparsers` method of `argparse.ArgumentParser` to enable sub-commands:
  ```py
  subparsers = parser.add_subparsers(
     dest="subcmd",
     required=True,
     title="List of sub-commands",
     description="For an overview of action specific parameters, use %(prog)s <SUB-COMMAND> --help",
     help="Sub-command help", metavar="<SUB-COMMAND>"
  )
  ```
2. For each sub-command, add a subparser to `subparsers`:
  ```py
  subparser = subparsers.add_parser("user", help="Manage users")
  ```
3. Add options to each subparser by using its `add_argument()` method, as you would do it for `argparse.ArgumentParser`
4. After a call to `args = parser.parse_args()`, `args.subcmd` will hold the name of the invoked sub-command as a `str`. `args` will also hold all values for options of the sub-command together with values for global options.

### Example

<div class="alert alert-block alert-info">
ℹ In the following example, we are going to use so called <a href="https://www.pythonstacks.com/blog/post/type-hints-python/">type hints</a>. We will use them as an aid for the reader understand which types a function expect as an argument and which type the function returns.
</div>
The following function `manage_user` expects a `str` as an argument and has no return value:

```py
def manage_user(user: str) -> None: ...
```

In [ ]:
%%writefile iam_example.py
#!/bin/env python3

import argparse

def manage_user(user: str) -> None:
    print(f"👤 {user}")

def manage_group(group: str) -> None:
    print(f"👥 {group}")

def main():
    parser = argparse.ArgumentParser(description="IAM tool")
    subparsers = parser.add_subparsers(
        dest="subcmd",
        required=True,
        title="List of sub-commands",
        description="For an overview of action specific parameters, use %(prog)s <SUB-COMMAND> --help",
        help="Sub-command help", metavar="<SUB-COMMAND>"
    )
    subparser_user = subparsers.add_parser("user", help="Manage users")
    subparser_user.add_argument("user", help="The user to manage")
    subparser_group = subparsers.add_parser("group", help="Manage groups")
    subparser_group.add_argument("group", help="The group to manage")

    args = parser.parse_args()
    if args.subcmd == "user":
        manage_user(args.user)
    elif args.subcmd == "group":
        manage_group(args.group)

if __name__ == "__main__":
    main()

In [ ]:
!chmod u+x iam_example.py

In [ ]:
%%script bash --no-raise-error

./iam_example.py

In [ ]:
%%script bash --no-raise-error

./iam_example.py --help

In [ ]:
%%script bash --no-raise-error

./iam_example.py user --help

In [ ]:
%%script bash --no-raise-error

./iam_example.py user chucknorris
./iam_example.py group roundhousekicks

### Pro Tipp: avoid if / else branching

```py
    ...
    subparser_user = subparsers.add_parser("user", help="Manage users")
    subparser_user.add_argument("user", help="The user to manage")
    subparser_group = subparsers.add_parser("group", help="Manage groups")
    subparser_group.add_argument("group", help="The group to manage")
    ...
```

After parsing the command line using `args = parser.parse_args()`, `args` will hold a string valued field `subcmd`.

We then have to compare this string against all possibilities 🤢:

```py
 if args.subcmd == "user": ...
 elif args.subcmd == "group": ...
 ... # maybe more to come?
```

**👍 Better**:

```py
    ...
    subparser_user = subparsers.add_parser("user", help="Manage users")
    subparser_user.set_defaults(subcmd=manage_user) # <- new
    subparser_user.add_argument("user", help="The user to manage")
    subparser_group = subparsers.add_parser("group", help="Manage group")
    subparser_group.set_defaults(subcmd=manage_group) # <- new
    subparser_group.add_argument("group", help="The group to manage")
    ...
```

After parsing the command line, `args.subcmd` will hold a function instead of a string (either `manage_user` or `manage_group`).
But we still need to either call `args.subcmd(args.user)` is `args.subcmd` is `manage_user` or `args.subcmd(args.group)` if `args.subcmd` is `manage_group`.

🥵 This still would require an `if` / `else` branching for the argument.

**👍👍 Even better**:

We can pass to `args.subcmd` all variables contained in `args` (except `subcmd`) as keyword arguments by using the `**` operator:

```py
args = parser.parse_args()
# turn `args` into a dict:
kwargs = vars(args)
# Extract the subcmd field from kwargs.
# As we dont want to pass `args.subcmd` to the functions (we dont want to pass the function to itself), we use `pop` to remove `subcmd` from the `dict` resulting from `vars()`:
subcmd = kwargs.pop("subcmd")
# Pass all remaining arguments to the function `subcmd`:
subcmd(**kwargs)
```

### Exercise 3

Write a CLI `user.py` with 2 sub-commands `add`, `remove` which calls the following 2 functions:

```py
def add(user: str, groups: list[str]) -> None:
    group_repr = " 👥 " + ",".join(groups) if groups else ""
    print(f"👤 {user}{group_repr}")

def remove(user: str) -> None:
    print(f"❌ {user}")
```

The command line should look like:
```sh
./user.py add "chucknorris" -g "admin" -g "superpower"
./user.py remove "bambi"
```

#### Tipps

* Use `action="append"` for the `-g` argument so it can be passed multiple times and appends each option to a list.
* Use the `dest` keyword in `add_argument()` so the value names in `args` match the argument names of the functions `add()`, `remove()`

In [ ]:
%%writefile ex3_solution.py
#!/bin/env python3

import argparse

def add(user: str, groups: list[str]) -> None:
    group_repr = " 👥 " + ",".join(groups) if groups else ""
    print(f"👤 {user}{group_repr}")

def remove(user: str) -> None:
    print(f"❌ {user}")

def main():
    parser = argparse.ArgumentParser(description="User tool")
    subparsers = parser.add_subparsers(
        dest="subcmd",
        required=True,
        title="List of sub-commands",
        description="For an overview of action specific parameters, use %(prog)s <SUB-COMMAND> --help",
        help="Sub-command help", metavar="<SUB-COMMAND>"
    )
    subparser_add = subparsers.add_parser("add", help="Add user")
    subparser_add.add_argument("user", help="The user to add")
    subparser_add.add_argument(
        "--group", "-g",
        dest="groups",
        action="append",
        help="Add user to a group (can be passed multiple times)"
    )
    subparser_add.set_defaults(subcmd=add)
    subparser_remove = subparsers.add_parser("remove", help="Remove user")
    subparser_remove.add_argument("user", help="The user to remove")
    subparser_remove.set_defaults(subcmd=remove)

    args = vars(parser.parse_args())
    subcmd = args.pop("subcmd")
    subcmd(**args)

if __name__ == "__main__":
    main()

In [ ]:
!chmod u+x ex3_solution.py

In [ ]:
%%script bash --no-raise-error

./ex3_solution.py

In [ ]:
%%script bash --no-raise-error

./ex3_solution.py add "chucknorris" -g "roundhouse-kicks" -g "can-count-to-infinity"
./ex3_solution.py remove "some_user"

## Trickier Topics

### Passwords

Sometimes you want to let the CLI prompt the user for a password.
Of course, you don't want to see the input on the screen.

With `argparse` this can be done by writing a custom [Action class](https://docs.python.org/3/library/argparse.html#action-classes) (with [click](https://click.palletsprojects.com/en/7.x/), this would be much easier).

In [ ]:
%%writefile auth.py
#!/bin/env python3

import argparse
import getpass

class PasswordPrompAction(argparse.Action):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, nargs=0, **kwargs)

    def __call__(self, parser, namespace, values, option_string=None):
        password = getpass.getpass(prompt='🔑: ')
        setattr(namespace, self.dest, password)
        
def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--password", "-p", action=PasswordPrompAction, help="Prompt for a password")
    args = parser.parse_args()
    print(f"Who needs privacy? Lets leak the password: {args.password}")


if __name__ == "__main__":
    main()

In [ ]:
!chmod u+x auth.py

You have to run this script from the command line, Jupyter has troubles reading passwords from a shell command. In Jupyter Lab, click on the `+` symbol right to the last tab and open a terminal

```sh
python ./auth.py -p
```

When resetting passwords, one often needs to enter the new password twice, in case you entered a typo. To show the password prompt a second time and compare the two inputs, we would replace our `__call__` according to:

In [ ]:
%%writefile auth_confirm.py
#!/bin/env python3

import argparse
import getpass

class PasswordPrompAction(argparse.Action):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, nargs=0, **kwargs)

    def __call__(self, parser, namespace, values, option_string=None):
        password = getpass.getpass(prompt='🔑: ')
        confirmation = getpass.getpass(prompt='🔑 (confirmation): ')
        if confirmation != password:
            raise argparse.ArgumentError(self, "💥 You entered two different passwords!")
        setattr(namespace, self.dest, password)
        
def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--password", "-p", action=PasswordPrompAction, help="Prompt for a password")
    args = parser.parse_args()
    print(f"Who needs privacy? Lets leak the password: {args.password}")


if __name__ == "__main__":
    main()

In [ ]:
!chmod u+x auth_confirm.py

```sh
python ./auth_confirm.py -p
```